# The Cutting Stock Problem
You are given a product with a standard length of `totalLength`.
This product (e.g., a steel coil) can be cut down to smaller lengths (`lengths`).
For these smaller lengths, you face deterministic demand (`demand`).

The task is to determine possible patterns of cutting the standard length product into pieces with smaller lengths so that
- the total scrap / waste is minimized or
- the number of standard length pieces used is minimized

Smaller length pieces that exceed demand can be stored, but there is only a limited storage capacity (`capacity`).

Note that the number of smaller lengths can be easily extended to > 2 (adjusting `lengths` and `demand`). 

In [4]:
# parameters
totalLength = 210
lengths = [45, 27]
demand = [200, 3000]  # demand for each length
capacity = 160  # capacity of storage for waste / remainings

num = len(lengths)

In [5]:
# determine maximum amount of pieces for each smaller length per standard length
max_ = {}
for i in range(0, num):
    max_[i] = int(totalLength/lengths[i])
    print(f"Maximum number of pieces for item {i}: {max_[i]}")

Maximum number of pieces for item 0: 4
Maximum number of pieces for item 1: 7


### Patterns
In a first step, we identify all possible patterns (i.e., combinations of cutting the standard length product into smaller length products).
For this, we calculate the total number of possible combinations / patterns (`numCombinations`) and map each of these patterns to a number between `0` and `numCombinations`.

In [38]:
multipliers = {}
for i in range(0, num):
    multipliers[i] = 1
for i in range(0, num-1):
    for j in range(i+1, num):
        multipliers[i] *= (max_[j]+1)

numCombinations = 1
for i in range(0, num):
    numCombinations *= max_[i]+1

print(f"Number of possible patterns: {numCombinations}")
    
patterns = []
for i in range(0, numCombinations):
    comb = ()
    rem = i
    for j in range(0, num):
        temp = int(rem/multipliers[j])
        rem = i % multipliers[j]
        comb = comb + (temp,)
    len_ = 0
    for i in range(0, num):
        len_ += comb[i]*lengths[i]
    patterns.append({
        "pattern": comb,
        "length": len_,
        "scrap": totalLength - len_
    })
print()
print("pattern\t | length\t | scrap")
for i in range(0, len(patterns)):
    p = patterns[i]
    print(f"{p['pattern']}\t | {p['length']}\t\t | {p['scrap']}")

Number of possible patterns: 40

pattern	 | length	 | scrap
(0, 0)	 | 0		 | 210
(0, 1)	 | 27		 | 183
(0, 2)	 | 54		 | 156
(0, 3)	 | 81		 | 129
(0, 4)	 | 108		 | 102
(0, 5)	 | 135		 | 75
(0, 6)	 | 162		 | 48
(0, 7)	 | 189		 | 21
(1, 0)	 | 45		 | 165
(1, 1)	 | 72		 | 138
(1, 2)	 | 99		 | 111
(1, 3)	 | 126		 | 84
(1, 4)	 | 153		 | 57
(1, 5)	 | 180		 | 30
(1, 6)	 | 207		 | 3
(1, 7)	 | 234		 | -24
(2, 0)	 | 90		 | 120
(2, 1)	 | 117		 | 93
(2, 2)	 | 144		 | 66
(2, 3)	 | 171		 | 39
(2, 4)	 | 198		 | 12
(2, 5)	 | 225		 | -15
(2, 6)	 | 252		 | -42
(2, 7)	 | 279		 | -69
(3, 0)	 | 135		 | 75
(3, 1)	 | 162		 | 48
(3, 2)	 | 189		 | 21
(3, 3)	 | 216		 | -6
(3, 4)	 | 243		 | -33
(3, 5)	 | 270		 | -60
(3, 6)	 | 297		 | -87
(3, 7)	 | 324		 | -114
(4, 0)	 | 180		 | 30
(4, 1)	 | 207		 | 3
(4, 2)	 | 234		 | -24
(4, 3)	 | 261		 | -51
(4, 4)	 | 288		 | -78
(4, 5)	 | 315		 | -105
(4, 6)	 | 342		 | -132
(4, 7)	 | 369		 | -159


In the next step, we identify valid patterns (total length does not exceed standard product length `totalLength`) and undominated patterns (it is not possible to cut an additional smaller length piece).

In [36]:
patterns_select = []
for p in patterns:
    p["valid"] = (p["length"] <= totalLength)
    p["undominated"] = True
    for i in range(0, num):
        p["undominated"] = p["undominated"] and (p["scrap"] < lengths[i])
    if p["valid"] and p["undominated"]:
        patterns_select.append(p)

print(f"Number of valid and undominated patterns: {len(patterns_select)}")
print()
print("pattern\t | length\t | scrap")
for i in range(0, len(patterns_select)):
    p = patterns_select[i]
    print(f"{p['pattern']}\t | {p['length']}\t\t | {p['scrap']}")

Number of valid and undominated patterns: 5

pattern	 | length	 | scrap
(0, 7)	 | 189		 | 21
(1, 6)	 | 207		 | 3
(2, 4)	 | 198		 | 12
(3, 2)	 | 189		 | 21
(4, 1)	 | 207		 | 3


### Optimization
Using `scipy`, we define the constraints and objective functions.

The decision variables consist of the number of times, a pattern is used, and the number of leftovers for each smaller length.

In [19]:
import numpy as np
from scipy import optimize

minScrap = []
minRolls = []
for p in range(0, len(patterns_select)):
    minScrap.append(patterns_select[p]["scrap"])
    minRolls.append(1)
for l in lengths:
    minScrap.append(0)  # no penalty for overstock
    minRolls.append(0)  # no penalty for overstock
    
# order fulfillment
A = []
b_lb = []
b_ub = []
for i in range(0, num):
    temp = []
    for p in range(0, len(patterns_select)):
        temp.append(patterns_select[p]["pattern"][i])
    for j in range(0, num):
        if i==j:
            temp.append(-1)  # consider overstock y
        else:
            temp.append(0)
    A.append(temp)
    b_ub.append(demand[i])
    b_lb.append(demand[i])
c1 = optimize.LinearConstraint(A, b_lb, b_ub)

# storage restriction
A = []
for p in range(0, len(patterns_select)):
    A.append(0)
for l in lengths:
    A.append(1)
c2 = optimize.LinearConstraint(A, 0, capacity)

constraints = [c1,c2]

bounds = optimize.Bounds(0, np.inf)
integrality = np.ones_like(minScrap)  # integers

Select the appropriate objective function

In [22]:
obj = minScrap
# obj = minRolls

Run the optimization

In [31]:
res = optimize.milp(c=obj, constraints=constraints, integrality=integrality, bounds=bounds)
print(res.message)
print(res.x)

print()
print("pattern\t | # of times in optimal solution")
for i in range(0, len(patterns_select)):
    p = patterns_select[i]
    print(f"{p['pattern']}\t | {res.x[i]}")

print()
print("length\t | demand\t | sum\t\t | leftovers")
for i in range(0, num):
    total = 0
    for p in range(0, len(patterns_select)):
        total += res.x[p]*patterns_select[p]["pattern"][i]
    print(f"{lengths[i]}\t | {demand[i]}\t\t | {total}\t | {total - demand[i]}")

print()
    
scrap = 0
for p in range(0, len(patterns_select)):
    scrap += res.x[p]*patterns_select[p]["scrap"]
print(f"scrap = {scrap}")

numRolls = 0
for p in range(0, len(patterns_select)):
    numRolls += res.x[p]
print(f"numRolls = {numRolls}")

Optimization terminated successfully. (HiGHS Status 7: Optimal)
[120. 360.   0.   0.   0. 160.   0.]

pattern	 | # of times in optimal solution
(0, 7)	 | 120.0
(1, 6)	 | 360.0
(2, 4)	 | 0.0
(3, 2)	 | 0.0
(4, 1)	 | 0.0

length	 | demand	 | sum		 | leftovers
45	 | 200		 | 360.0	 | 160.0
27	 | 3000		 | 3000.0	 | 0.0

scrap = 3600.0
numRolls = 480.0
